In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random
import matplotlib.pyplot as plt
class Autoencoder(nn.Module):
    def __init__(self, encoding_dims):
        super(Autoencoder, self).__init__()
        
        # Encoder layers
        self.encoder_layers = nn.ModuleList()
        input_dim = 784
        for dim in encoding_dims:
            self.encoder_layers.append(nn.Linear(input_dim, dim))
            input_dim = dim
        
        # Decoder layers
        self.decoder_layers = nn.ModuleList()
        for dim in reversed(encoding_dims[:-1]):
            self.decoder_layers.append(nn.Linear(input_dim, dim))
            input_dim = dim
        self.decoder_layers.append(nn.Linear(input_dim, 784))
        
    def forward(self, x):
        # Encoder
        for layer in self.encoder_layers:
            x = F.tanh(layer(x))
        
        # Decoder
        for layer in self.decoder_layers[:-1]:
            x = F.tanh(layer(x))
        x = F.tanh(self.decoder_layers[-1](x))
        return x

def test_mnist_pytorch():
    # Define transformations and load MNIST dataset
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create autoencoder model
    autoencoder = Autoencoder([256, 64]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    
    # Training loop
    for epoch in range(10):
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.view(data.size(0), -1).to(device)
            optimizer.zero_grad()
            outputs = autoencoder(data)
            loss = criterion(outputs, data)
            loss.backward()
            optimizer.step()
    
    # Generate reconstructions
    test_images, _ = next(iter(test_loader))
    test_images = test_images.view(test_images.size(0), -1).to(device)
    decoded_images = autoencoder(test_images).cpu().detach().numpy()
    test_images = test_images.cpu().numpy()
    
    # Plot the original and reconstructed images
    plt.figure(figsize=(20, 4))
    random.seed(42)
    random_test_images = random.sample(range(0, 1000), 10)
    for i, image_idx in enumerate(random_test_images):
        # plot original image
        ax = plt.subplot(2, 10, i + 1)
        plt.imshow(test_images[image_idx].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # plot reconstructed image
        ax = plt.subplot(2, 10, 10 + i + 1)
        plt.imshow(decoded_images[image_idx].reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

test_mnist_pytorch()
